In [1]:
# ============================================================================
# GOLD LAYER - BUSINESS REPORTING TABLES
# Purpose: Create normalized star schema for Power BI semantic model
# ============================================================================

from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime, timedelta

print("=" * 70)
print("GOLD LAYER - BUSINESS REPORTING TABLES")
print("=" * 70)

# Load denormalized table
denormalized = spark.table("Silver.dbo.silver_retailer_transactions")

# ============================================================================
# DIMENSION TABLES
# ============================================================================

print("\n" + "="*70)
print("CREATING DIMENSION TABLES")
print("="*70)

# ============================================================================
# 1. DIM_DATE (Date Dimension)
# ============================================================================

print("\n1. Creating dim_date...")

# Generate date range (2024-01-01 to 2025-12-31)
from pyspark.sql.types import DateType

date_df = spark.range(
    0, 
    (datetime(2025, 12, 31) - datetime(2024, 1, 1)).days + 1
).select(
    F.expr("date_add('2024-01-01', cast(id as int))").alias("date")
)

dim_date = date_df \
    .withColumn("date_key", F.date_format(F.col("date"), "yyyyMMdd").cast("int")) \
    .withColumn("year", F.year("date")) \
    .withColumn("quarter", F.quarter("date")) \
    .withColumn("month", F.month("date")) \
    .withColumn("month_name", F.date_format("date", "MMMM")) \
    .withColumn("week", F.weekofyear("date")) \
    .withColumn("day_of_month", F.dayofmonth("date")) \
    .withColumn("day_of_week", F.dayofweek("date")) \
    .withColumn("day_name", F.date_format("date", "EEEE")) \
    .withColumn("is_weekend", F.when(F.col("day_of_week").isin([1, 7]), True).otherwise(False)) \
    .withColumn("is_month_end", 
                F.when(F.col("day_of_month") == F.dayofmonth(F.last_day("date")), True).otherwise(False)) \
    .withColumn("fiscal_year", 
                F.when(F.col("month") >= 4, F.col("year")).otherwise(F.col("year") - 1)) \
    .withColumn("fiscal_quarter", 
                F.when(F.col("month").between(4, 6), 1)
                 .when(F.col("month").between(7, 9), 2)
                 .when(F.col("month").between(10, 12), 3)
                 .otherwise(4))

dim_date.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_date")

print(f"✓ Created dim_date with {dim_date.count():,} dates")

# ============================================================================
# 2. DIM_RETAILERS (Retailer Dimension - SCD Type 2)
# ============================================================================

print("\n2. Creating dim_retailers...")

# Get unique retailers
dim_retailers = denormalized \
    .select(
        "retailer_id",
        "business_name",
        "owner_name",
        "owner_gender",
        "owner_age",
        "shop_type",
        "state",
        "urbanization_level",
        "latitude",
        "longitude",
        "years_in_business",
        "months_in_business",
        "num_employees",
        "has_business_registration",
        "mobile_money_pattern",
        "monthly_mobile_money_txns",
        "credit_segment",
        "credit_limit",
        "account_age_days"
    ).dropDuplicates(["retailer_id"])

# Add surrogate key and SCD Type 2 fields
dim_retailers = dim_retailers \
    .withColumn("retailer_key", F.monotonically_increasing_id()) \
    .withColumn("valid_from", F.lit(datetime(2024, 1, 1))) \
    .withColumn("valid_to", F.lit(datetime(2099, 12, 31))) \
    .withColumn("is_current", F.lit(True))

dim_retailers.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_retailers")

print(f"✓ Created dim_retailers with {dim_retailers.count():,} retailers")

# ============================================================================
# 3. DIM_GEOGRAPHY
# ============================================================================

print("\n3. Creating dim_geography...")

dim_geography = denormalized \
    .select("state", "urbanization_level") \
    .dropDuplicates() \
    .withColumn("geography_key", F.monotonically_increasing_id()) \
    .withColumn("region", 
                F.when(F.col("state").isin(["Lagos", "Ogun", "Oyo", "Osun"]), "South West")
                 .when(F.col("state").isin(["Rivers", "Delta", "Edo"]), "South South")
                 .when(F.col("state").isin(["Anambra", "Enugu", "Imo"]), "South East")
                 .when(F.col("state").isin(["Kano", "Kaduna", "Katsina"]), "North West")
                 .when(F.col("state").isin(["FCT", "Niger", "Kwara"]), "North Central")
                 .otherwise("Other"))

dim_geography.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_geography")

print(f"✓ Created dim_geography with {dim_geography.count():,} locations")

# ============================================================================
# 4. DIM_PRODUCTS
# ============================================================================

print("\n4. Creating dim_products...")

# Explode product categories and manufacturers
products_exploded = denormalized \
    .select(
        F.explode(F.col("product_categories_array")).alias("product_category"),
        F.explode(F.col("manufacturers_array")).alias("manufacturer")
    ).dropDuplicates()

dim_products = products_exploded \
    .withColumn("product_key", F.monotonically_increasing_id()) \
    .withColumn("category_group", 
                F.when(F.col("product_category").isin(["Beverages", "Confectionery", "Snacks"]), "Food & Beverages")
                 .when(F.col("product_category").isin(["Personal Care", "Household Items"]), "Personal Care")
                 .otherwise("Other"))

dim_products.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_products")

print(f"✓ Created dim_products with {dim_products.count():,} products")

# ============================================================================
# 5. DIM_CREDIT_SEGMENTS
# ============================================================================

print("\n5. Creating dim_credit_segments...")

dim_credit_segments = spark.createDataFrame([
    ("Excellent", 800, 850, 0.002, "Platinum"),
    ("Good", 750, 799, 0.008, "Gold"),
    ("Fair", 700, 749, 0.025, "Silver"),
    ("Moderate Risk", 650, 699, 0.06, "Bronze"),
    ("High Risk", 550, 649, 0.15, "Copper"),
], ["segment_name", "min_score", "max_score", "default_rate_target", "risk_tier"])

dim_credit_segments = dim_credit_segments \
    .withColumn("segment_key", F.monotonically_increasing_id())

dim_credit_segments.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_credit_segments")

print(f"✓ Created dim_credit_segments with {dim_credit_segments.count():,} segments")

# ============================================================================
# 6. DIM_SHOP_TYPES
# ============================================================================

print("\n6. Creating dim_shop_types...")

dim_shop_types = spark.createDataFrame([
    ("Superette", 4, "High"),
    ("Mini Mart", 3, "Medium-High"),
    ("Provision Store", 2, "Medium"),
    ("Market Stall", 1, "Low"),
    ("Kiosk", 0, "Very Low"),
], ["shop_type", "shop_type_encoded", "formality_level"])

dim_shop_types = dim_shop_types \
    .withColumn("shop_type_key", F.monotonically_increasing_id())

dim_shop_types.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_shop_types")

print(f"✓ Created dim_shop_types with {dim_shop_types.count():,} types")

# ============================================================================
# FACT TABLES
# ============================================================================

print("\n" + "="*70)
print("CREATING FACT TABLES")
print("="*70)

# ============================================================================
# 7. FACT_TRANSACTIONS (Main Fact Table)
# ============================================================================

print("\n7. Creating fact_transactions...")

# Join with dimensions to get keys
fact_transactions = denormalized \
    .join(
        dim_retailers.select("retailer_id", "retailer_key"),
        "retailer_id",
        "left"
    ) \
    .withColumn("date_key", F.date_format(F.col("order_date"), "yyyyMMdd").cast("int"))

fact_transactions = fact_transactions.select(
    # Keys (for joins)
    "transaction_id",
    "retailer_key",
    "date_key",
    "repayment_id",
    
    # Degenerate dimensions (high cardinality)
    "order_number",
    "business_name",
    
    # Dates
    "order_date",
    "due_date",
    "payment_date",
    "order_year",
    "order_month",
    "order_quarter",
    
    # Measures (Additive)
    F.col("order_amount").cast("decimal(18,2)").alias("order_amount"),
    F.col("amount_paid").cast("decimal(18,2)").alias("amount_paid"),
    F.col("days_late").cast("int").alias("days_late"),
    F.col("num_products").cast("int").alias("num_products"),
    
    # Calculated measures
    (F.col("order_amount") - F.col("amount_paid")).cast("decimal(18,2)").alias("payment_deficit"),
    F.col("payment_completion_rate").cast("decimal(5,4)").alias("payment_completion_rate"),
    
    # Flags (for filtering/grouping)
    "payment_status",
    "is_on_time",
    "is_defaulted",
    "is_first_order",
    "is_late_payment",
    
    # Context attributes
    "credit_segment",
    "shop_type",
    "state",
    "urbanization_level",
)

fact_transactions.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("fact_transactions")

print(f"✓ Created fact_transactions with {fact_transactions.count():,} transactions")


   


StatementMeta(, 57ecd1dc-5cf8-4736-8698-336081b2a402, 3, Finished, Available, Finished)

GOLD LAYER - BUSINESS REPORTING TABLES

CREATING DIMENSION TABLES

1. Creating dim_date...
✓ Created dim_date with 731 dates

2. Creating dim_retailers...
✓ Created dim_retailers with 10,000 retailers

3. Creating dim_geography...
✓ Created dim_geography with 33 locations

4. Creating dim_products...
✓ Created dim_products with 100 products

5. Creating dim_credit_segments...
✓ Created dim_credit_segments with 5 segments

6. Creating dim_shop_types...
✓ Created dim_shop_types with 5 types

CREATING FACT TABLES

7. Creating fact_transactions...
✓ Created fact_transactions with 75,347 transactions


In [2]:
date = spark.table("dim_date")

StatementMeta(, 57ecd1dc-5cf8-4736-8698-336081b2a402, 4, Finished, Available, Finished)

In [3]:
display(date)

StatementMeta(, 57ecd1dc-5cf8-4736-8698-336081b2a402, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 91e8c06d-7348-4850-997f-4df456532bb0)

In [1]:
# ============================================================================
# 🥇 GOLD LAYER - BUSINESS TABLES + ML INTEGRATION
# Purpose: Integrate ML risk scores into star schema for Power BI
# ============================================================================

from pyspark.sql import functions as F
from pyspark.sql.window import Window

print("=" * 70)
print("🥇 GOLD LAYER - INTEGRATING ML SCORES INTO STAR SCHEMA")
print("=" * 70)

# Load existing tables
fact_transactions = spark.table("fact_transactions")
dim_retailers = spark.table("dim_retailers")
risk_scores = spark.table("ml_retailer_scores_final")

# ============================================================================
# OPTION 1: CREATE DIM_RISK_SCORES (Separate Dimension)
# ============================================================================
print("\n1. Creating dim_risk_scores (separate dimension)...")

dim_risk_scores = risk_scores.select(
    "retailer_id",
    "risk_probability",
    "predicted_risk_segment"
) \
    .withColumn("risk_score_key", F.monotonically_increasing_id()) \
    .withColumn("risk_tier_encoded", 
                F.when(F.col("predicted_risk_segment") == "Unrated / New", 0)
                 .when(F.col("predicted_risk_segment") == "Low Risk", 1)
                 .when(F.col("predicted_risk_segment") == "Medium Risk", 2)
                 .when(F.col("predicted_risk_segment") == "High Risk", 3)
                 .when(F.col("predicted_risk_segment") == "Confirmed Default", 4)
                 .otherwise(0)) \
    .withColumn("is_high_risk", 
                F.when(F.col("predicted_risk_segment").isin(["High Risk", "Confirmed Default"]), True)
                 .otherwise(False))

dim_risk_scores.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("dim_risk_scores")

print(f"✓ Created dim_risk_scores with {dim_risk_scores.count():,} records")

# ============================================================================
# OPTION 2: ADD CURRENT RISK SCORE TO DIM_RETAILERS
# ============================================================================
print("\n2. Adding current risk score reference to dim_retailers...")

dim_retailers_with_risk = dim_retailers \
    .join(
        dim_risk_scores.select("retailer_id", "risk_score_key", "predicted_risk_segment", "risk_probability"),
        "retailer_id",
        "left"
    ) \
    .withColumn("current_risk_score_key", F.col("risk_score_key")) \
    .withColumn("current_risk_segment", F.col("predicted_risk_segment")) \
    .withColumn("current_risk_probability", F.col("risk_probability")) \
    .drop("risk_score_key", "predicted_risk_segment", "risk_probability")

dim_retailers_with_risk.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("dim_retailers")

print("✓ Updated dim_retailers with current risk score reference")

# ============================================================================
# OPTION 3: FACT TABLE WITH ALL FOREIGN KEYS (Geo, Shop, Credit)
# ============================================================================
print("\n3. Creating fact_transactions_with_risk (with ALL Foreign Keys)...")

# ----------------------------------------------------------------------------
# STEP A: LOAD & PREPARE DIMENSIONS (The Sources of Truth)
# ----------------------------------------------------------------------------
print("   -> Generating dynamic dimensions (Geography, Shop Type)...")

# 1. GEOGRAPHY DIMENSION
#    We create this dynamically from the distinct states in your retailer list
dim_geography = dim_retailers.select("state").distinct().orderBy("state") \
    .withColumn("geography_key", F.monotonically_increasing_id() + 1)

dim_geography.write.format("delta").mode("overwrite").saveAsTable("dim_geography")

# 2. SHOP TYPE DIMENSION
#    Same here, distinct shop types become a dimension
dim_shop_type = dim_retailers.select("shop_type").distinct().orderBy("shop_type") \
    .withColumn("shop_type_key", F.monotonically_increasing_id() + 1)

dim_shop_type.write.format("delta").mode("overwrite").saveAsTable("dim_shop_type")

# 3. CREDIT SEGMENT DIMENSION (Existing)
#    We assume 'dim_credit_segments' exists.
dim_credit_segments = spark.table("dim_credit_segments")


# ----------------------------------------------------------------------------
# STEP B: CREATE THE "MASTER KEY LOOKUP"
# ----------------------------------------------------------------------------
# We join 'dim_retailers' to all these dimensions to get the keys for every retailer.
# This creates a "Rosetta Stone" that maps a Retailer Key -> All Integer Keys.

print("   -> Mapping Retailers to Keys...")

retailer_keys_map = dim_retailers_with_risk.alias("r") \
    .join(dim_geography.alias("g"), F.col("r.state") == F.col("g.state"), "left") \
    .join(dim_shop_type.alias("s"), F.col("r.shop_type") == F.col("s.shop_type"), "left") \
    .join(dim_credit_segments.alias("c"), F.col("r.credit_segment") == F.col("c.segment_name"), "left") \
    .select(
        F.col("r.retailer_key"),
        F.col("r.current_risk_score_key"),
        F.col("r.current_risk_segment"),
        F.col("r.current_risk_probability"),
        # The New Keys
        F.col("g.geography_key"),
        F.col("s.shop_type_key"),
        F.col("c.segment_key").alias("credit_segment_key")
    )

# ----------------------------------------------------------------------------
# STEP C: STAMP THE FACT TABLE
# ----------------------------------------------------------------------------
print("   -> Stamping Foreign Keys onto Fact Table...")

fact_transactions_with_risk = (
    fact_transactions
    .join(retailer_keys_map, "retailer_key", "left")
    .withColumn("risk_segment_at_score",
        # Auto-correct logic: If Model says Unrated but they are buying -> Low Risk
        F.when((F.col("current_risk_segment") == "Unrated / New"), "Low Risk")
         .otherwise(F.col("current_risk_segment"))
    )
    .withColumn("risk_probability_at_score", F.col("current_risk_probability"))
    .withColumn("is_high_risk_transaction",
        F.when(F.col("risk_segment_at_score").isin(["High Risk", "Confirmed Default"]), True)
         .otherwise(False)
    )
    .select(
        fact_transactions["*"],
        
        # ALL FOREIGN KEYS
        F.col("current_risk_score_key").alias("risk_score_key"),
        F.col("credit_segment_key"),
        F.col("geography_key"),
        F.col("shop_type_key"),
        
        # ML INSIGHTS
        F.col("risk_segment_at_score"),
        F.col("risk_probability_at_score"),
        F.col("is_high_risk_transaction")
    )
)

# ----------------------------------------------------------------------------
# STEP D: SAVE
# ----------------------------------------------------------------------------
fact_transactions_with_risk.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("fact_transactions_with_risk")

print(f"✓ Created fact_transactions_with_risk")
print(f"✓ Added Keys: geography_key, shop_type_key, credit_segment_key")
print(f"✓ Created Dimensions: dim_geography, dim_shop_type")


# ============================================================================
# CREATE ANALYTICAL VIEWS FOR POWER BI
# ============================================================================

print("\n4. Creating analytical views...")

# View 1: High Risk Transactions Summary
high_risk_transactions = fact_transactions_with_risk \
    .filter(F.col("is_high_risk_transaction") == True) \
    .groupBy(
        "risk_segment_at_score",
        "order_year",
        "order_month"
    ).agg(
        F.count("transaction_id").alias("num_transactions"),
        F.sum("order_amount").alias("total_order_value"),
        F.sum("amount_paid").alias("total_amount_paid"),
        F.avg("days_late").alias("avg_days_late"),
        F.sum(F.when(F.col("is_defaulted"), 1).otherwise(0)).alias("num_defaults")
    ) \
    .withColumn("default_rate", F.col("num_defaults") / F.col("num_transactions"))

high_risk_transactions.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold_high_risk_transactions_summary")

print("✓ Created gold_high_risk_transactions_summary")

# View 2: Risk Segment Performance
risk_segment_performance = fact_transactions_with_risk \
    .groupBy("risk_segment_at_score").agg(
        F.count("transaction_id").alias("total_transactions"),
        F.countDistinct("retailer_key").alias("num_retailers"),
        F.sum("order_amount").alias("total_revenue"),
        F.avg("order_amount").alias("avg_transaction_value"),
        F.sum("amount_paid").alias("total_collected"),
        F.avg("days_late").alias("avg_days_late"),
        F.sum(F.when(F.col("is_defaulted"), 1).otherwise(0)).alias("num_defaults")
    ) \
    .withColumn("collection_rate", F.col("total_collected") / F.col("total_revenue")) \
    .withColumn("default_rate", F.col("num_defaults") / F.col("total_transactions"))

risk_segment_performance.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold_risk_segment_performance")

print("✓ Created gold_risk_segment_performance")

# View 3: At-Risk Retailers (Active but High Risk)
at_risk_retailers = dim_retailers \
    .join(
        fact_transactions.groupBy("retailer_key").agg(
            F.max("order_date").alias("last_order_date"),
            F.sum("order_amount").alias("lifetime_value")
        ),
        "retailer_key",
        "left"
    ) \
    .filter(
        (F.col("current_risk_segment").isin(["High Risk", "Medium Risk"])) &
        (F.datediff(F.current_date(), F.col("last_order_date")) <= 60)  # Active in last 60 days
    ) \
    .select(
        "retailer_key",
        "retailer_id",
        "business_name",
        "credit_segment",
        "current_risk_segment",
        "current_risk_probability",
        "credit_limit",
        "last_order_date",
        "lifetime_value"
    ) \
    .orderBy(F.col("lifetime_value").desc())

at_risk_retailers.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold_at_risk_retailers")

print("✓ Created gold_at_risk_retailers")

# ============================================================================
# VALIDATION & INSIGHTS
# ============================================================================

print("\n" + "=" * 70)
print("VALIDATION & INSIGHTS")
print("=" * 70)

# 1. How many transactions are from high-risk retailers?
high_risk_count = fact_transactions_with_risk.filter(F.col("is_high_risk_transaction") == True).count()
total_count = fact_transactions_with_risk.count()
high_risk_pct = high_risk_count / total_count * 100

print(f"\nHigh-Risk Transaction Analysis:")
print(f"  Total transactions: {total_count:,}")
print(f"  High-risk transactions: {high_risk_count:,} ({high_risk_pct:.1f}%)")

# 2. Revenue exposure by risk segment
revenue_by_risk = fact_transactions_with_risk.groupBy("risk_segment_at_score") \
    .agg(F.sum("order_amount").alias("revenue")) \
    .orderBy(F.col("revenue").desc())

print(f"\nRevenue Exposure by Risk Segment:")
revenue_df = revenue_by_risk.toPandas()
total_revenue = revenue_df["revenue"].sum()
for _, row in revenue_df.iterrows():
    segment = row["risk_segment_at_score"] if row["risk_segment_at_score"] else "Unknown"
    revenue = row["revenue"]
    pct = revenue / total_revenue * 100 if total_revenue > 0 else 0
    print(f"  {segment}: ₦{revenue:,.0f} ({pct:.1f}%)")

# 3. Default rate by risk segment
default_analysis = fact_transactions_with_risk.groupBy("risk_segment_at_score") \
    .agg(
        F.count("*").alias("transactions"),
        F.sum(F.when(F.col("is_defaulted"), 1).otherwise(0)).alias("defaults")
    ) \
    .withColumn("default_rate", F.col("defaults") / F.col("transactions"))

print(f"\nActual Default Rate by Predicted Risk Segment:")
print("(Validates if ML model predictions match reality)")
default_df = default_analysis.toPandas()
for _, row in default_df.iterrows():
    segment = row["risk_segment_at_score"] if row["risk_segment_at_score"] else "Unknown"
    rate = row["default_rate"]
    print(f"  {segment}: {rate:.2%}")

# 4. Verify Keys
print("\nVerifying New Foreign Keys (Sample):")
fact_transactions_with_risk.select("transaction_id", "geography_key", "shop_type_key", "credit_segment_key").show(5)

print("\n" + "=" * 70)
print("✓ SCRIPT COMPLETE")
print("=" * 70)

StatementMeta(, 96a8ea16-75b5-4201-90f0-31b30e242b70, 3, Finished, Available, Finished)

🥇 GOLD LAYER - INTEGRATING ML SCORES INTO STAR SCHEMA

1. Creating dim_risk_scores (separate dimension)...
✓ Created dim_risk_scores with 10,000 records

2. Adding current risk score reference to dim_retailers...
✓ Updated dim_retailers with current risk score reference

3. Creating fact_transactions_with_risk (with ALL Foreign Keys)...
   -> Generating dynamic dimensions (Geography, Shop Type)...
   -> Mapping Retailers to Keys...
   -> Stamping Foreign Keys onto Fact Table...
✓ Created fact_transactions_with_risk
✓ Added Keys: geography_key, shop_type_key, credit_segment_key
✓ Created Dimensions: dim_geography, dim_shop_type

4. Creating analytical views...
✓ Created gold_high_risk_transactions_summary
✓ Created gold_risk_segment_performance
✓ Created gold_at_risk_retailers

VALIDATION & INSIGHTS

High-Risk Transaction Analysis:
  Total transactions: 75,347
  High-risk transactions: 219 (0.3%)

Revenue Exposure by Risk Segment:
  Low Risk: ₦910,704,991 (99.0%)
  Medium Risk: ₦6,800,9

In [2]:
df = spark.sql("SELECT * FROM Gold.dbo.ml_retailer_scores_final LIMIT 1000")
display(df)

StatementMeta(, 46affa9e-9854-44cd-8130-da3922074690, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 62b1da65-d67f-40b8-867d-eea5499f91bf)

In [4]:
# =========================================================
# 🕵️‍♂️ DISK CHECK: IS THE SAVED TABLE STALE?
# =========================================================

print("Checking 'ml_retailer_scores_final' on disk...")

check_df = spark.sql("""
    SELECT 
        predicted_risk_segment, 
        COUNT(*) as count, 
        SUM(total_orders) as total_orders_check
    FROM 
        ml_retailer_scores_final
    WHERE 
        predicted_risk_segment = 'Unrated / New'
    GROUP BY 
        predicted_risk_segment
""")

display(check_df)

StatementMeta(, 46affa9e-9854-44cd-8130-da3922074690, 6, Finished, Available, Finished)

Checking 'ml_retailer_scores_final' on disk...


SynapseWidget(Synapse.DataFrame, d5bfbbc6-5550-49c5-803e-5b97f555354a)